# Advanced Lane Finding

The main scope of the project is to program an advanced version of lane finding using camera calibration to remove distortion, apply diverse technique to isolate the lane marks and warp to have a "bird's-eye view" of the street.

### Goals

- Compute camera calibration matrix and distortion cohefficients using given chessboard images
- Apply correction for distortion
- Use color transformation, gradients and other techniques to create thresholded binary images
- Create a "bird-eye view" of the image trhough perspective transformation
- Find lane pixels and fit to determine lane boundaries
- Calculate the curvature of the lane and the vehicole position
- Project (unwrap) the detected lane boundaries back to the original image
- Display lane boundaries

## Imports

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

## Camera calibration
Camera calibration using OpenCV function.
The camera_calibration method takes in input 4 parameters, namely:
- *pathre* : a path where calibration images are stored with a regular expression to filter the files to use
- *nx* : number of reference point over the x axis
- *ny* : number of reference point over the y axis
- *save_calib* : if True a picle file named calibration_data.p will be created storing the matrix and distortion coefficients for later use ( reference: [CarND-Camera-Calibration](https://github.com/udacity/CarND-Camera-Calibration) )

In [2]:
def undistort(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)

In [3]:
'''
Generate camera calibration matrix and distortion coefficients.
takes in input 4 parameters, namely:

pathre : a path where calibration images are stored with a regular expression to filter the files to use
nx : number of reference point over the x axis
ny : number of reference point over the y axis
save_calib : if True a picle file named calibration_data.p will be created storing the matrix 
and distortion coefficients for later use ( reference: CarND-Camera-Calibration )
'''
def camera_calibration(pathre="camera_cal/calibration*.jpg", nx=8, ny=6, save_calib=True):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((ny*nx,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob(pathre)
    img_size = None
    # Step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img_size is None:
            img_size = (img.shape[1], img.shape[0])
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    if save_calib ==True:
        dist_pickle = {}
        dist_pickle["mtx"] = mtx
        dist_pickle["dist"] = dist
        pickle.dump( dist_pickle, open( "calibration_data.p", "wb" ) )
        img = cv2.imread(images[0])
        undistorted=undistort(img, mtx, dist)
        cv2.imwrite('output_images/test_distorted.jpg', img)
        cv2.imwrite('output_images/test_undistorted.jpg', undistorted)
    return mtx, dist
        
mtx, dist = camera_calibration(pathre="camera_cal/calibration*.jpg", nx=9, ny=6, save_calib=True)

## Pipeline method definitions. 
Some methods could be simple recall to OpenCV methoe( e.g. undistort )

### Distortion correction
Wrapper method to cv2.undistort

### Creation of thresholded binary images
Here I start with the methods already used and developed during the lessons, so:
- Color and gradient using the luminsity and saturation channel combination

In [4]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [5]:
def color_gradient(img, s_thresh=(170, 255), sx_thresh=(20, 100), save_output=False):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    h_channel = hls[:,:,0]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    sobelxh = cv2.Sobel(h_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelxh = np.absolute(sobelxh) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobelh = np.uint8(255*abs_sobelx/np.max(abs_sobelxh))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold x gradient hue
    sxbinaryh = np.zeros_like(scaled_sobelh)
    sxbinaryh[(scaled_sobelh >= 50) & (scaled_sobelh <= 70)] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.zeros_like(s_channel)
    color_binary[(sxbinaryh==1) | (sxbinary == 1) | (s_binary==1)] =255
#     vertices = np.array([[(570,445),(220,700),(1135,700),(730,445),(650,445),(995,680),(280,680),(630,445)]])
#     color_binary_masked = region_of_interest(color_binary, vertices)
    if save_output == True:
        cv2.imwrite('output_images/color_grad_input.jpg', img)
        cv2.imwrite('output_images/color_grad_out.jpg', color_binary)
#         cv2.imwrite('output_images/color_grad_out_mask.jpg', color_binary_masked)
    return color_binary

### Warp to create bird-eye view

In [6]:
def warp(img, save_output=False):
    gray = img.copy()
    src = np.float32([[600,465],[250,700],[1125,700],[720,465]])
    dst = np.float32([[250,0],[250,img.shape[0]],[1060,img.shape[0]],[1060,0]])
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, (img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)
    
#     vertices = np.array([[(250,0),(220,700),(1135,700),(730,445),(650,445),(995,680),(280,680),(630,445)]])
#     color_binary_masked = region_of_interest(color_binary, vertices)
    
    if save_output == True:
        cv2.imwrite('output_images/warp_input.jpg', img)
        cv2.imwrite('output_images/warp_out.jpg', warped)
    return warped, M, Minv

def unwarp(img, Minv, save_output=False):
    unwarped = cv2.warpPerspective(img, Minv, (img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)
    if save_output == True:
        cv2.imwrite('output_images/unwarp_input.jpg', img)
        cv2.imwrite('output_images/unwarp_out.jpg', unwarped)
    return unwarped

In [7]:
# HYPERPARAMETERS
# Choose the number of sliding windows
nwindows = 9
# Set the width of the windows +/- margin
margin = 100
# Set minimum number of pixels found to recenter window
minpix = 50

lanewd_mt = 3.7
lanewd_px = (1060-250) #/lanewd_mt

def find_lane_pixels(binary_warped, save_output=False):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
#     detected_lanewd = (rightx_base-leftx_base)
#     if detected_lanewd/lanewd_px < 0.8 or detected_lanewd/lanewd_px > 1.2:
#         print("-"*20)
#         print(f"!!!{detected_lanewd/lanewd_px} - {detected_lanewd} - {lanewd_px}")
#         print(leftx_base, rightx_base)
#         if midpoint - leftx_base < lanewd_px*0.4 :
#             print("mod_leftx")
#             leftx_base = np.argmax(histogram[:leftx_base-10])
#         elif rightx_base - midpoint < lanewd_px*0.4 :
#             rightx_base = np.argmax(histogram[rightx_base+10:])
#         print(leftx_base, rightx_base)
#         print("-"*20)

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current -margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
#         cv2.rectangle(out_img,(win_xleft_low,win_y_low),
#         (win_xleft_high,win_y_high),(0,255,0), 2) 
#         cv2.rectangle(out_img,(win_xright_low,win_y_low),
#         (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &(nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    if save_output == True:
        cv2.imwrite('output_images/findlanes_input.jpg', binary_warped)
        cv2.imwrite('output_images/findlanes_out.jpg', out_img)
    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(binary_warped, save_output=False):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped, save_output=save_output)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='red')
    window_img = np.zeros_like(out_img)

    left_line_window = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    right_line_window = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    lane_pts=np.hstack((left_line_window, right_line_window))
    cv2.fillPoly(window_img, np.int_([lane_pts]), (0,255, 0))
    out_img = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    if save_output == True:
        cv2.imwrite('output_images/fit_polynomial_input.jpg', binary_warped)
        cv2.imwrite('output_images/fit_polynomial_out.jpg', out_img)
    return out_img, left_fit, right_fit, left_fitx, right_fitx

# out_img = fit_polynomial(cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY))
# plt.imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY))

In [8]:
def fit_poly(img_shape, leftx, lefty, rightx, righty, prevleft_fit, prevright_fit):
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    if prevleft_fit.shape[0]>1:
        i0=np.max([0, prevleft_fit.shape[0]-7])
        tmp = np.append(prevleft_fit[i0:-1], left_fit).reshape(-1,3)
        left_fit = np.mean(tmp, axis=0)
    if prevright_fit.shape[0]>1:
        i0=np.max([0, prevright_fit.shape[0]-7])
        tmp = np.append(prevright_fit[i0:-1], right_fit).reshape(-1,3)
        right_fit = np.mean(tmp, axis=0)
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    return left_fitx, right_fitx, ploty, left_fit, right_fit

def search_around_poly(binary_warped, prevleft_fit, prevright_fit, save_output=False):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    if prevleft_fit is None or prevright_fit is None:
        return fit_polynomial(binary_warped, save_output=save_output)
    left_fit = prevleft_fit[-1]
    right_fit = prevright_fit[-1]
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    detected_lanewd = (np.mean(rightx)-np.mean(leftx))
    if detected_lanewd/lanewd_px < 0.8 or detected_lanewd/lanewd_px > 1.2 or \
        len(leftx)==0 or len(lefty)==0 or len(rightx)==0 or len(righty)==0:
        return fit_polynomial(binary_warped, save_output=save_output)
    left_fitx, right_fitx, ploty, left_fit, right_fit = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty, 
                                                                 prevleft_fit, prevright_fit)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    
    left_line_window = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    right_line_window = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    
    
    lane_pts=np.hstack((left_line_window, right_line_window))
    
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([lane_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    if save_output == True:
        cv2.imwrite('output_images/search_around_poly_input.jpg', binary_warped)
        cv2.imwrite('output_images/search_around_poly_out.jpg', result)
    return result, left_fit, right_fit, left_fitx, right_fitx

In [9]:
prevleft_fit=None
prevright_fit=None

In [10]:
def measure_curvature(img_shape, leftx, rightx, xm_per_pix=3.7/700, ym_per_pix = 30/720):
    ploty = np.linspace(0, img_shape[0] - 1, num=img_shape[0])
    
    rleftx = leftx[::-1]  # Reverse to match top-to-bottom in y
    rrightx = rightx[::-1]  # Reverse to match top-to-bottom in y
    
    # Fit a second order polynomial to pixel positions in each lane line
    rleft_fit = np.polyfit(ploty, rleftx, 2)
    rleft_fitx = rleft_fit[0]*ploty**2 + rleft_fit[1]*ploty + rleft_fit[2]
    rright_fit = np.polyfit(ploty, rrightx, 2)
    rright_fitx = rright_fit[0]*ploty**2 + rright_fit[1]*ploty + rright_fit[2]

    # Fit new polynomials to x,y in world space
    y_eval = np.max(ploty)
    left_fit_cr = np.polyfit(ploty*ym_per_pix, rleftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rrightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # Now our radius of curvature is in meters
    return (left_curverad, right_curverad)

In [11]:
def measure_offset(img_shape, leftx, rightx, xm_per_pix=3.7/700):
    ## Image mid horizontal position 
    midpoint = img_shape[1]//2
        
    ## Car position with respect to the lane
    car_position = (leftx[-1] + rightx[-1])//2
    
    ## Horizontal car offset 
    offsetx = (midpoint - car_position) * xm_per_pix

    return offsetx

In [12]:
def display_info(img, left_curverad, right_curverad, offset):    
    out_img = img.copy()
    cv2.putText(out_img, 'Left lane curvature: {:.2f} m'.format(left_curverad), 
                (60, 60), cv2.FONT_HERSHEY_COMPLEX, 1.5, (255,255,255), 5)
    cv2.putText(out_img, 'Right lane curvature: {:.2f} m'.format(right_curverad), 
                (60, 110), cv2.FONT_HERSHEY_COMPLEX, 1.5, (255,255,255), 5)
    
    # Display car offset
    cv2.putText(out_img, 'Car offset: {:.2f} m'.format(offset), 
                (60, 160), cv2.FONT_HERSHEY_COMPLEX, 1.5, (255,255,255), 5)
    
    return out_img

In [13]:
save_output=True
def pipeline(image):
    global prevleft_fit, prevright_fit
    undistorted = undistort(image, mtx, dist)
    grad = color_gradient(undistorted, s_thresh=(170, 255), sx_thresh=(35, 95), save_output=save_output)
    warped,_, Minv = warp(grad, save_output=save_output)
    out_img, left_fit, right_fit, left_x, right_x = search_around_poly(warped,
                                                                       prevleft_fit, prevright_fit, save_output=save_output)
    if prevleft_fit is None:
        prevleft_fit = []
    
    prevleft_fit = np.append(prevleft_fit, left_fit).reshape(-1,3)
    
    if prevright_fit is None:
        prevright_fit = []
    prevright_fit = np.append(prevright_fit, right_fit).reshape(-1,3)
        
    left_curverad, right_curverad = measure_curvature(warped.shape, left_x, right_x, xm_per_pix=3.7/lanewd_px, ym_per_pix = 30/warped.shape[0])
    offset = measure_offset(warped.shape, left_x, right_x, xm_per_pix=3.7/lanewd_px)
    newwarp = unwarp(out_img, Minv) #cv2.warpPerspective(warped, Minv, (image.shape[1], image.shape[0])) 
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    resinfo= display_info(result, left_curverad, right_curverad, offset)
    return resinfo

In [14]:
import matplotlib.image as mpimg
prevleft_fit=None
prevright_fit=None
image = mpimg.imread("test_images/test1.jpg")
t = pipeline(image)
t2 = pipeline(image)
t2 = pipeline(image)
t2 = pipeline(image)
cv2.imwrite('output_images/pipeline_input.jpg', image)
cv2.imwrite('output_images/firstrun_out.jpg', t)
cv2.imwrite('output_images/secondrun_out.jpg', t2)

True

In [15]:
prevleft_fit=None
prevright_fit=None
save_output=False
test_output = 'output_images/project_out.mp4'
clip1 = VideoFileClip("project_video.mp4")

# test_output = "tests/out_challenge_video.mp4"
# clip1 = VideoFileClip("challenge_video.mp4").subclip(0,5)

white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(test_output, audio=False)

t:   0%|          | 3/1260 [00:00<01:09, 18.18it/s, now=None]

Moviepy - Building video output_images/project_out.mp4.
Moviepy - Writing video output_images/project_out.mp4



Moviepy - Done !
Moviepy - video ready output_images/project_out.mp4
Wall time: 1min 45s


In [16]:
prevleft_fit=None
prevright_fit=None
save_output=False
test_output = 'output_images/challenge_video.mp4'
clip1 = VideoFileClip("challenge_video.mp4")

# test_output = "tests/out_challenge_video.mp4"
# clip1 = VideoFileClip("challenge_video.mp4").subclip(0,5)

white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(test_output, audio=False)

t:   1%|          | 3/485 [00:00<00:26, 18.07it/s, now=None]

Moviepy - Building video output_images/challenge_video.mp4.
Moviepy - Writing video output_images/challenge_video.mp4



Moviepy - Done !
Moviepy - video ready output_images/challenge_video.mp4
Wall time: 40.9 s


In [17]:
prevleft_fit=None
prevright_fit=None
save_output=False
test_output = 'output_images/harder_challenge_video.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")

# test_output = "tests/out_challenge_video.mp4"
# clip1 = VideoFileClip("challenge_video.mp4").subclip(0,5)

white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(test_output, audio=False)

t:   0%|          | 3/1199 [00:00<01:08, 17.54it/s, now=None]

Moviepy - Building video output_images/harder_challenge_video.mp4.
Moviepy - Writing video output_images/harder_challenge_video.mp4



Moviepy - Done !
Moviepy - video ready output_images/harder_challenge_video.mp4
Wall time: 1min 58s
